In [1]:
import numpy as np

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

In [2]:
torch.manual_seed(1)

In [3]:
def split(examples):
    return [(s.split(), l) for s, l in examples]

In [4]:
data = split([
    ('me gusta comer en la cafeteria', 'SPANISH'),
    ('Give it to me', 'ENGLISH'),
    ('No creo que sea una buena idea', 'SPANISH'),
    ('No it is not a good idea to get lost at sea', 'ENGLISH')
])

In [5]:
test_data = split([
    ('Yo creo que si', 'SPANISH'),
    ('it is lost on me', 'ENGLISH')
])

In [6]:
word_to_index = {}
for sentence, _ in data + test_data:
    for word in sentence:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
print(word_to_index)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [7]:
label_to_index = {'SPANISH': 0, 'ENGLISH': 1}

In [8]:
VOCAB_SIZE = len(word_to_index)
NUM_LABELS = 2

In [9]:
class BoWClassifier(nn.Module):
    
    def __init__(self, num_labels, vocab_size):
        super().__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [10]:
def make_bow_vector(sentence, word_to_index):
    vec = torch.zeros(len(word_to_index))
    for word in sentence:
        vec[word_to_index[word]] += 1
    return vec.view(1, -1)

In [11]:
def make_target(label, label_to_index):
    return torch.LongTensor([label_to_index[label]])

In [12]:
v = make_bow_vector('not a good sea'.split(), word_to_index)
v.shape

torch.Size([1, 26])

In [13]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [14]:
def test(model, dataset):
    with torch.no_grad():
        for instance, label in dataset:
            bow_vec = make_bow_vector(instance, word_to_index)
            log_probs = model(bow_vec)
            print(instance, log_probs)

In [15]:
test(model, test_data)

['Yo', 'creo', 'que', 'si'] tensor([[-0.6250, -0.7662]])
['it', 'is', 'lost', 'on', 'me'] tensor([[-0.5870, -0.8119]])


In [16]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    epoch_loss = 0
    
    for instance, label in data:
        model.zero_grad()
        bow_vec = make_bow_vector(instance, word_to_index)
        target = make_target(label, label_to_index)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    if epoch % 20 == 19:
        print('Epoch: %03d - loss: %2.4f' % (epoch + 1, epoch_loss))

Epoch: 020 - loss: 0.1962
Epoch: 040 - loss: 0.0954
Epoch: 060 - loss: 0.0629
Epoch: 080 - loss: 0.0469
Epoch: 100 - loss: 0.0374


In [17]:
test(model, test_data)

['Yo', 'creo', 'que', 'si'] tensor([[-0.1210, -2.1721]])
['it', 'is', 'lost', 'on', 'me'] tensor([[-2.7767, -0.0643]])


## Word Embeddings: Encoding Lexical Semantics

In [19]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

In [20]:
torch.manual_seed(1)

In [21]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [22]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [23]:
trigrams = [
    ([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
     for i in range(len(test_sentence) - 2)]

In [24]:
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [25]:
vocab = set(test_sentence)
word_to_index = {word: i for i, word in enumerate(vocab)}

In [26]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        hid = F.relu(self.linear1(embeds))
        out = self.linear2(hid)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [29]:
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    
    for context, target in trigrams:
        context_idxs = torch.tensor([
            word_to_index[w] for w in context], 
            dtype=torch.long)
        
        target = torch.tensor(
            [word_to_index[target]], 
            dtype=torch.long)
        
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print('Epoch %03d loss: %2.4f' % (epoch + 1, total_loss))

Epoch 001 loss: 519.5103
Epoch 002 loss: 517.1770
Epoch 003 loss: 514.8590
Epoch 004 loss: 512.5547
Epoch 005 loss: 510.2629
Epoch 006 loss: 507.9822
Epoch 007 loss: 505.7097
Epoch 008 loss: 503.4464
Epoch 009 loss: 501.1916
Epoch 010 loss: 498.9447


## Exercise: Continuous Bag-of-Words

In [30]:
CONTEXT_SIZE = 2

In [31]:
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [32]:
vocab = set(raw_text)
vocab_size = len(vocab)

In [33]:
word_to_index = {word: i for i, word in enumerate(vocab)}

In [42]:
def create_dataset(text, context_size):
    cs = context_size
    for i in range(cs, len(text) - cs):
        left = [text[i - c] for c in range(cs, 0, -1)]
        right = [text[i + c] for c in range(1, cs + 1)]
        context = left + right
        target = text[i]
        yield context, target

In [45]:
dataset = create_dataset(raw_text, CONTEXT_SIZE)

In [ ]:
class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, 128)
    
    def forward(self, inputs):
        embed = self.embedding(inputs).view((1, -1))
        out = self.linear(torch.sum(embed))
        log_probs = F.log_softmax(out)
        return log_probs